In [1]:
import sys
sys.path.insert(1, r"/vol/bitbucket/mjh24/IAEA-thesis")
from Stage1.GAT.GATModel import GraphAttentionNetwork
from Single_Website_Download.Download import main
import torch
from Stage1.ExtractingGraphs.HTMLtoGraph import html_to_graph, EdgeFeatures
from Stage1.ExtractingGraphs.seleniumDriver import *
from Stage1.ExtractingLabels.swde_label_extraction import label_extraction, _find_matches
from Stage2.textExtractor import extract_chunk_xpaths
from Stage1.tree_helpers import *
from pathlib import Path
from scipy import sparse
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
#Import model
model = GraphAttentionNetwork(in_dim = 114, edge_in_dim = 200, edge_emb_dim = 32, hidden1 = 32, hidden2 = 32, hidden3 = 8, heads = 2)
state_dict = torch.load("../../Stage1/GAT/FULLTRAINEDALLDATAModelf1-75-learning.pt", map_location=torch.device(device))
model.load_state_dict(state_dict)
model.to(device)

GraphAttentionNetwork(
  (tr1): TransformerConv(114, 32, heads=2)
  (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (tr2): TransformerConv(64, 32, heads=1)
  (ln2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (tr3): TransformerConv(32, 8, heads=1)
  (edge_mlp): Sequential(
    (0): Linear(in_features=48, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [3]:
# url = "C:\\Users\\micha\\Documents\\Imperial Courses\\Thesis\\IAEA-thesis\\data\\swde\\sourceCode\\sourceCode\\movie\\movie\\movie-allmovie(2000)\\0000.htm"
url = r"https://www.nucnet.org/news/parliament-resolution-paves-way-for-establishing-nuclear-energy-legislation-6-4-2024"
htmlFile = Path("/vol/bitbucket/mjh24/IAEA-thesis/data/websites/test.html")
main(url,1,htmlFile)

Saved https://www.nucnet.org/news/parliament-resolution-paves-way-for-establishing-nuclear-energy-legislation-6-4-2024 → /vol/bitbucket/mjh24/IAEA-thesis/data/websites/test.html
Page limit hit


In [4]:
driver_init(True)
restart_Driver(True)
_, X_npy, E_sparse, edge_index, bbox = html_to_graph(htmlFile, get_Driver())
quit_driver()
X_sparse = sparse.csr_matrix(X_npy)
E_sparse = sparse.csr_matrix(E_sparse)

In [5]:
def get_all_candidate_edges(filepath, safeurl, X_npy):
    xpaths = extract_chunk_xpaths(filepath, safeurl)
    tree = load_html_as_tree(filepath)
    node2index, index2node = bfs_index_map(tree)

    XPaths: Dict[etree._Element, str] = {}
    xp2node: Dict[str, etree._Element] = {}
    for el in node2index:
        xp = tree.getpath(el)
        XPaths[el] = xp
        xp2node[xp] = el
    parentMap, depthMap = build_parent_and_depth_maps(tree)

    txtNodes = []
    coords = []

    for xp in xpaths:
        node = xp2node[xp]
        txtNodes.append(node2index[node])

    #This logic should make an undirected graph
    for i in txtNodes:
        for j in txtNodes:
            if i != j:
                coords.append((i,j))
    
    features = []
    for i, j in coords:
        features.append(EdgeFeatures(
            edgeStart=i,
            edgeEnd=j,
            edgeStartNode=index2node[i],
            edgeEndNode=index2node[j],
            X=X_npy,
            bboxs=bbox,
            parentMap=parentMap,
            depthMap=depthMap,
            XPaths=XPaths
        ))

    return np.array(coords), features

# def get_all_candidate_edges(filepath, X_npy):
#     TXTLENGTH = 400
#     tree = load_html_as_tree(filepath)
#     node2index, index2node = bfs_index_map(tree)
#     txtNodes = []
#     coords = []

#     stack = [tree.getroot()]
#     while stack:
#         #limit to nodes with text less than a certain length. once that node has been read, do not grab its children
#         node = stack.pop()
#         txtlen = len(get_node_text(node))

#         if txtlen > 2 and txtlen < TXTLENGTH:
#             txtNodes.append(node2index[node])
#             continue  # skip this node's children entirely

#         # Reverse so we process in document order.
#         for child in reversed(list(node)):
#             stack.append(child)

#     #This logic should make an undirected graph
#     for i in txtNodes:
#         for j in txtNodes:
#             if i != j:
#                 coords.append((i,j))

#     XPaths: Dict[etree._Element, str] = {}
#     for el in node2index:
#         xp = tree.getpath(el)
#         XPaths[el] = xp
#     parentMap, depthMap = build_parent_and_depth_maps(tree)
    
#     features = []
#     for i, j in coords:
#         features.append(EdgeFeatures(
#             edgeStart=i,
#             edgeEnd=j,
#             edgeStartNode=index2node[i],
#             edgeEndNode=index2node[j],
#             X=X_npy,
#             bboxs=bbox,
#             parentMap=parentMap,
#             depthMap=depthMap,
#             XPaths=XPaths
#         ))

#     return np.array(coords), features

def get_specific_candidate_edges(filepath, txt, X_npy):
    TXTLENGTH = 400
    tree = load_html_as_tree(filepath)
    node2index, index2node = bfs_index_map(tree)
    parentMap, depthMap = build_parent_and_depth_maps(tree)
    node = _find_matches(tree, txt, depthMap)[0]
    nodeindex = node2index[node]
    
    coords = []

    #This logic should make an undirected graph
    for n, _ in iter_elements_with_direct_text(tree):
        i = node2index[n]
        if i != nodeindex:
            coords.append((i,nodeindex))
            coords.append((nodeindex, i))

    XPaths: Dict[etree._Element, str] = {}
    for el in node2index:
        xp = tree.getpath(el)
        XPaths[el] = xp
    
    features = []
    for i, j in coords:
        features.append(EdgeFeatures(
            edgeStart=i,
            edgeEnd=j,
            edgeStartNode=index2node[i],
            edgeEndNode=index2node[j],
            X=X_npy,
            bboxs=bbox,
            parentMap=parentMap,
            depthMap=depthMap,
            XPaths=XPaths
        ))

    return np.array(coords), features

In [6]:
label_index, label_features = get_all_candidate_edges(htmlFile, url, X_npy)
label_index = np.array(label_index)
label_features = sparse.csr_matrix(label_features)

In [7]:
def _npz_to_csr(csr: sparse.csr, dtype=torch.float32):
    crow = torch.from_numpy(csr.indptr.astype(np.int64))
    col  = torch.from_numpy(csr.indices.astype(np.int64))
    val  = torch.from_numpy(csr.data).to(dtype)
    return torch.sparse_csr_tensor(
        crow, col, val, size=csr.shape, dtype=dtype, requires_grad=False
    )

def _npy_to_tensor(arr: np.ndarray, dtype=torch.long):
    return torch.from_numpy(arr).to(dtype).t().contiguous()

X, Aei, Aef, Lei, Lef = _npz_to_csr(X_sparse), _npy_to_tensor(edge_index), _npz_to_csr(E_sparse), _npy_to_tensor(label_index), _npz_to_csr(label_features)
X, Aei, Aef, Lei, Lef = X.to(device), Aei.to(device), Aef.to(device), Lei.to(device), Lef.to(device)
print(Lef.shape)

torch.Size([5550, 200])


/tmp/ipykernel_3711900/3318138968.py:5: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(


In [8]:
model.eval()
logits = model(X, Aei, Aef, Lei, Lef)
probs  = torch.sigmoid(logits)
probs.shape

torch.Size([5550])

In [13]:
order = np.argsort(probs.squeeze().tolist())[::-1]
sorted_label_index = label_index[order]

# Build a mapping from tuple to its position for fast lookup
pair_to_pos = {tuple(pair): idx for idx, pair in enumerate(sorted_label_index)}
avg_pos = []
for idx, label in enumerate(sorted_label_index):
    pair = tuple(label)
    rev_pair = (label[1], label[0])
    rev_idx = pair_to_pos.get(rev_pair, idx)
    avg_pos.append((idx + rev_idx) / 2)

order = np.argsort(avg_pos)
sorted_label_index = sorted_label_index[order]
sorted_label_index

array([[ 77, 134],
       [134,  77],
       [130,  77],
       ...,
       [ 35, 247],
       [ 35, 186],
       [186,  35]], shape=(5550, 2))

In [14]:
tree = load_html_as_tree(htmlFile)
_, index2node = bfs_index_map(tree)

for label in sorted_label_index:
    print(get_node_text(index2node[label[0]], True).strip(), "->", get_node_text(index2node[label[1]], True).strip())
    #print(tree.getpath(index2node[label[0]]), "->", tree.getpath(index2node[label[1]]))

tags -> regulator
regulator -> tags
estonia -> tags
tags -> estonia
login -> independentnuclearnews
independentnuclearnews -> login
tags -> law
law -> tags
resolution -> tags
tags -> resolution
login -> joinnucnet
parliament -> tags
tags -> parliament
joinnucnet -> login
newbuild -> mostpopular
mostpopular -> newbuild
mostpopular -> newbuild
newbuild -> mostpopular
nuclearpolitics -> mostpopular
mostpopular -> nuclearpolitics
mostpopular -> newbuild
newbuild -> mostpopular
tags -> newbuild
newbuild -> tags
mostpopular -> corporate
corporate -> mostpopular
tags -> newbuild
newbuild -> tags
nuclearpolitics -> tags
loadmore -> related
related -> loadmore
tags -> nuclearpolitics
newbuild -> mostpopular
mostpopular -> newbuild
login -> related
related -> login
newbuild -> tags
tags -> newbuild
corporate -> tags
tags -> corporate
related -> smallmodularreactors
smallmodularreactors -> related
related -> newbuild
newbuild -> related
nuclearpolitics -> related
related -> nuclearpolitics
relate

1) see how many external links I am picking up, if its an acceptable amount, dw about it
filter out links in html that point to external webpages

2) compare this to just putting it all into an llm. make this a prompting model